<a href="https://colab.research.google.com/github/pablolake/pacman/blob/main/tfg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install rlcard
!pip install rlcard.utils

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.5/265.5 KB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rlcard: filename=rlcard-1.1.0-py3-none-any.whl size=322200 sha256=942fd5f22fedfb48631d16ebf30d3bd1c477dd4fb0c6882543fb8c3f218ed017
  Stored in directory: /root/.cache/pip/wheels/1d/a8/66/5a7a7be9d7d0607ec8cabde164f3b44694800f394736ce56a4
Successfully built rlcard


In [ ]:

# Set up the game environment
import rlcard
from rlcard.agents import NolimitholdemHumanAgent as HumanAgent
from rlcard.envs import Env
from rlcard.games.nolimitholdem import Game
from rlcard.games.nolimitholdem.round import Action
import tensorflow as tf
from rlcard.utils import print_card
from rlcard.utils import (
    get_device,
    set_seed,
    tournament,
    reorganize,
    Logger,
    plot_curve,
)
from collections import namedtuple
from rlcard.agents.dqn_agent import DQNAgent

Transition = namedtuple('Transition', ['state', 'action', 'reward', 'next_state', 'done', 'legal_actions'])
# Set up the game environment
env = rlcard.make('no-limit-holdem')

# Create the human agent object
human_agent = HumanAgent(env.num_actions)

replay_memory_size = 20000
replay_memory_init_size = 1000
update_target_estimator_every = 1000
discount_factor = 0.99
epsilon_start = 1.0
epsilon_end = 0.1
epsilon_decay_steps = 20000
batch_size = 32
num_actions = env.num_actions
state_shape=env.state_shape[0]
train_every = 1
mlp_layers = [128, 128]
learning_rate = 0.001
device = get_device()

# Create the DQN agent object
dqn_agent = DQNAgent(
    replay_memory_size=replay_memory_size,
    replay_memory_init_size=replay_memory_init_size,
    update_target_estimator_every=update_target_estimator_every,
    discount_factor=discount_factor,
    epsilon_start=epsilon_start,
    epsilon_end=epsilon_end,
    epsilon_decay_steps=epsilon_decay_steps,
    batch_size=batch_size,
    num_actions=num_actions,
    state_shape=state_shape,
    train_every=train_every,
    mlp_layers=mlp_layers,
    learning_rate=learning_rate,
    device=device
)


# Reset the environment
state, _ = env.reset()

env.set_agents([human_agent, dqn_agent])


while (True):
    print(">> Start a new game")

    trajectories, payoffs = env.run(is_training=False)
    # If the human does not take the final action, we need to
    # print other players action
    final_state = trajectories[0][-1]
    action_record = final_state['action_record']
    state = final_state['raw_obs']
    _action_list = []
    for i in range(1, len(action_record)+1):
        if action_record[-i][0] == state['current_player']:
            break
        _action_list.insert(0, action_record[-i])
    for pair in _action_list:
        print('>> Player', pair[0], 'chooses', pair[1])

    # Let's take a look at what the agent card is
    print('===============     Cards all Players    ===============')
    for hands in env.get_perfect_information()['hand_cards']:
        print("hola",num_actions)
        print_card(hands)

    print('===============     Result     ===============')
    if payoffs[0] > 0:
        print('You win {} chips!'.format(payoffs[0]))
    elif payoffs[0] == 0:
        print('It is a tie.')
    else:
        print('You lose {} chips!'.format(-payoffs[0]))
    print('')

    input("Press any key to continue...")

# Nueva sección